In [5]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os


In [6]:
RANDOM_SEED = 42

In [7]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

In [8]:
wf = pd.read_csv("data/wf.csv")
city_yb = pd.read_csv("data/city_yb.csv")

wf["temp2"] = wf["temp"] ** 2
wf["l_aqi"] = np.log(1 + wf["aqi"])
wf["l_pm"] = np.log(1 + wf["pm"])
wf2020 = wf[(wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)].dropna(
    subset = ['aqi', 'pm']
)
wf2020['cities'] = wf2020['city_code'].astype('category')
wf2020['days'] = wf2020['daynum'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

fixed = ['treat']
for col in wf2020.columns:
    if 'cities' in col or 'days' in col:
        fixed.append(col)
        
weather = ['prec', 'snow', 'temp', 'temp2']
out = ["aqi", "l_aqi", "pm", "l_pm"]

## DOUBLE ML

In [9]:
treated = wf2020[wf2020['treat'] == 1]
treated = treated[['daynum', 'city_code']].groupby('city_code')
first = treated.apply(lambda x: x.sort_values(by = 'daynum', ascending=True).head(1))

day, count = np.unique(first.daynum, return_counts = True)
treat_day = day[count == max(count)][0]

num_cities = {d:c for d,c in zip(day, count)}
first = {city:day for day, city in first.values}

In [10]:
wf2020 = wf2020.assign(first = [first.get(city, 0) for city in wf2020['city_code']])
group = wf2020[(wf2020['first'] == treat_day) | (wf2020['first'] == 0)]
wf2020['first'] = wf2020['first'].astype('category')
wf2020 = pd.get_dummies(wf2020, drop_first=True)

In [11]:
group['pre'] = group['daynum'] < treat_day
group = group.groupby(['city_code', 'pre']).mean().reset_index('pre')

In [12]:
compact = group[~group['pre']]
aqi = group['aqi'].values
compact['Y1-Y0'] = aqi[~group['pre']] - aqi[group['pre']]

/Users/jtf/anaconda3/envs/mamba-base-env/envs/nn_class/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
compact = compact.reset_index()

In [14]:
# format this in a manner sympatico with ATT estimation

outcome = compact['Y1-Y0']
treatment = compact['treat']
confounders = compact[weather]

In [15]:
# specify a model for the conditional expected outcome

# TODO(victorveitch) the covariates have basically no predictive power, replace this example with something better

# make a function that returns a sklearn model for later use in k-folding
def make_Q_model():
    return LinearRegression()
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=100, max_depth=5)
Q_model = make_Q_model()

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict(X_test)

test_mse=mean_squared_error(y_pred, y_test)
print(f"Test MSE of fit model {test_mse}") 
baseline_mse=mean_squared_error(y_train.mean()*np.ones_like(y_test), y_test)
print(f"Test MSE of no-covariate model {baseline_mse}")

Test MSE of fit model 592.0016742435274
Test MSE of no-covariate model 763.2330734709204


In [16]:
# specify a model for the propensity score

def make_g_model():
    return LogisticRegression(max_iter=1000)
    return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100, max_depth=5)

g_model = make_g_model()
# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_train, X_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
g_model.fit(X_train, a_train)
a_pred = g_model.predict_proba(X_test)[:,1]

test_ce=log_loss(a_test, a_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.2074859981907707
Test CE of no-covariate model 0.23372561765023045


In [17]:
# helper functions to implement the cross fitting

def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]

    assert np.isnan(predictions).sum() == 0
    return predictions


def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model()
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [18]:
g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=10)

In [19]:
Q0,Q1=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="continuous")

In [20]:
data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
data_and_nuisance_estimates.head()

,g,Q0,Q1,A,Y
0,0.135846,-24.575467,-41.087033,1.0,-47.483196
1,0.094061,-17.032831,-34.925517,0.0,-38.480972
2,0.100673,-15.995116,-32.506683,0.0,-67.323789
3,0.094208,-15.429095,-34.433198,0.0,-37.060072
4,0.091899,-25.442115,-43.514097,0.0,-1.856384


In [21]:
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    # Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    """

    if prob_t is None:
        prob_t = A.mean() # estimate marginal probability of treatment

    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    scores = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    n = Y.shape[0] # number of observations
    std_hat = np.std(scores) / np.sqrt(n)

    return tau_hat, std_hat


In [22]:
tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is -19.319572199045226 pm 11.349616379309929


In [23]:
# for comparison, the point estimate without any covariate correction
outcome[treatment==1].mean()-outcome[treatment==0].mean()

-18.886275852614084

## Multiple time periods

In [24]:
wf2020 = wf2020.assign(first = [first.get(city, 0) for city in wf2020['city_code']])
wf2020['A'] = (wf2020['daynum'] == wf2020['first']).astype('int64')

wf2020['diff'] = wf2020.sort_values(by = 'daynum')['aqi'] \
            - wf2020.sort_values(by = 'daynum').groupby('city_code')['aqi'].shift(1)
wf2020 = wf2020.dropna(subset= ['diff'])

outcome = wf2020['diff']
treatment = wf2020['A']
confounders = wf2020[['first'] + weather]

In [25]:
# specify a model for the conditional expected outcome

# TODO(victorveitch) the covariates have basically no predictive power, replace this example with something better

# make a function that returns a sklearn model for later use in k-folding
def make_Q_model():
    #return LinearRegression()
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators = 100, max_depth=10)
Q_model = make_Q_model()

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict(X_test)

test_mse=mean_squared_error(y_pred, y_test)
print(f"Test MSE of fit model {test_mse}") 
baseline_mse=mean_squared_error(y_train.mean()*np.ones_like(y_test), y_test)
print(f"Test MSE of no-covariate model {baseline_mse}")

Test MSE of fit model 1090.0377218848553
Test MSE of no-covariate model 1165.2400531127032


In [26]:
# specify a model for the propensity score

def make_g_model():
    #return LogisticRegression(max_iter=1000)
    return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=100, max_depth=3)

g_model = make_g_model()
# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_train, X_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
g_model.fit(X_train, a_train)
a_pred = g_model.predict_proba(X_test)[:,1]

test_ce=log_loss(a_test, a_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.018717674730204158
Test CE of no-covariate model 0.024077153205500912


In [27]:
a_pred.min()

0.00043236041379117717

In [28]:
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    # Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    """

    if prob_t is None:
        prob_t = A.mean() # estimate marginal probability of treatment

    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    scores = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    n = Y.shape[0] # number of observations
    std_hat = np.std(scores) / np.sqrt(n)

    return tau_hat, std_hat

In [29]:
# for comparison, the point estimate without any covariate correction
outcome[treatment==1].mean()-outcome[treatment==0].mean()

-11.684548868980686

In [30]:
res = dict()
Q_model.fit(X_w_treatment, outcome)
g_model.fit(confounders, treatment)

RandomForestClassifier(max_depth=3, random_state=42)

In [31]:
for day in np.unique(wf2020['daynum']):
    df = wf2020[wf2020['daynum'] == day]
    outcome_t = df['diff']
    treatment_t = df['A']
    confounders_t = df[['first'] + weather]
    
    if df['A'].sum() == 0 or num_cities[day] < 2:
        continue
    
    X1 = confounders_t.copy()
    X0 = confounders_t.copy()
    X1["treatment"] = 1
    X0["treatment"] = 0
    
    Q0 = Q_model.predict(X0)
    Q1 = Q_model.predict(X1)
    g = g_model.predict_proba(confounders_t)[:,1]
    
    est, sd = att_aiptw(Q0, Q1, g, treatment_t, outcome_t)
    res[day] = (est, sd)
    
inv_var = np.array([1/v**2 for p,v in res.values()])
point = np.array([p for p,v in res.values()])    

tau_hat = (point * inv_var).sum()/inv_var.sum()
std_hat = np.sqrt(1/inv_var.sum())

print('%0.3f pm %0.3f' % (tau_hat, std_hat))

-4.916 pm 1.542


### Parallel Trends

In [ ]:
out_var = 'aqi'
pd.options.mode.chained_assignment = None  # default='warn' # NOTE: is this a bad?
# trends across time periods (days)
time_periods = np.sort(np.unique(wf2020["daynum"]))
pairwise_time_diffs = []

for t in range(0, len(time_periods)-1):
    wf_pt_post = wf2020[wf2020["daynum"] == time_periods[t+1]]
    outcome_t = wf_pt_post['diff']
    treatment_t = wf_pt_post['A']
    confounders_t = wf_pt_post[['first'] + weather]
    X1 = confounders_t.copy()
    X0 = confounders_t.copy()
    X1["treatment"] = 1
    X0["treatment"] = 0
    
    Q0 = Q_model.predict(X0)
    Q1 = Q_model.predict(X1)
    print(time_periods[t+1], np.mean(Q1 - Q0))

In [ ]:
# create week coefficient 
wf2020["week_coef"] = np.floor((wf2020["daynum"] - wf2020["first"])/7).astype(int)
# set -1 lead and untreated to NaN so they don't get week0 dummy
wf2020["week_coef"] = np.where((wf2020["week_coef"] == -1), np.NaN, wf2020["week_coef"])
wf2020["week_coef"][wf2020["first"] == 0] = np.NaN
wf2020["week_coef"] = wf2020["week_coef"].astype('category')
wf2020 = pd.get_dummies(wf2020)

week_coef = []
for col in wf2020.columns:
    if 'week_coef' in col:
        week_coef.append(col)

In [43]:
pairwise_time_diffs = []

for w in range(0, len(week_coef)-1):
    wf_pt_pre = wf2020[wf2020[week_coef[w]] == 1 ]
    wf_pt_post = wf2020[wf2020[week_coef[w+1]] == 1]
    outcome_t = wf_pt_post['diff']
    treatment_t = wf_pt_post['A']
    confounders_t = wf_pt_post[['first'] + weather]
    X1 = confounders_t.copy()
    X0 = confounders_t.copy()
    X1["treatment"] = 1
    X0["treatment"] = 0
    
    Q0 = Q_model.predict(X0)
    Q1 = Q_model.predict(X1)
    print(week_coef[w+1], np.mean(Q1) - np.mean(Q0))
    
    

week_coef_-7.0 -0.3364145862151906
week_coef_-6.0 -2.93730435177273
week_coef_-5.0 -1.2571279636109294
week_coef_-4.0 -2.7465710435608894
week_coef_-3.0 -1.7802101708449336
week_coef_-2.0 -2.670474698963522
week_coef_0.0 -2.3305858196633773
week_coef_1.0 -1.6426142976755265
week_coef_2.0 -0.9044038526144369
week_coef_3.0 -1.50629645110028
week_coef_4.0 -0.8348632231018416
week_coef_5.0 -0.6930900445352839


In [40]:
wf2020[week_coef]

,week_coef_-8.0,week_coef_-7.0,week_coef_-6.0,week_coef_-5.0,week_coef_-4.0,week_coef_-3.0,week_coef_-2.0,week_coef_0.0,week_coef_1.0,week_coef_2.0,week_coef_3.0,week_coef_4.0,week_coef_5.0
366,0,0,1,0,0,0,0,0,0,0,0,0,0
367,0,0,1,0,0,0,0,0,0,0,0,0,0
368,0,0,1,0,0,0,0,0,0,0,0,0,0
369,0,0,1,0,0,0,0,0,0,0,0,0,0
370,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144413,0,0,0,0,0,0,0,0,0,0,0,0,0
144414,0,0,0,0,0,0,0,0,0,0,0,0,0
144415,0,0,0,0,0,0,0,0,0,0,0,0,0
144416,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
# check parallel trends 
# E[Y_{t+1} - Y_{t} | A=1, X] - E[Y_{t+1} - Y_{t} | A=0, X] = 0 for pre-treatment periods

wf_pt = wf[(wf["daynum"] >= 8401) & (wf["daynum"]<= 8461)].dropna(
    subset = ['aqi', 'pm']
)
wf_pt['cities'] = wf_pt['city_code'].astype('category')
wf_pt['days'] = wf_pt['daynum'].astype('category')
wf_pt = pd.get_dummies(wf_pt, drop_first=True)

wf_pt = wf_pt.assign(first = [first.get(city, 0) for city in wf_pt['city_code']])
wf_pt = wf_pt[(wf_pt['first'] == treat_day) | (wf_pt['first'] == 0)]
# wf_pt['first'] = wf_pt['first'].astype('category')
#wf_pt = pd.get_dummies(wf_pt, drop_first=True)


out_var = 'aqi'
pd.options.mode.chained_assignment = None  # default='warn' # NOTE: is this a bad?
# trends across time periods (days)
time_periods = np.sort(np.unique(wf_pt["daynum"]))
pairwise_time_diffs = []
for t in range(0, len(time_periods)-1):
    wf_pt_pre = wf_pt[wf_pt["daynum"] == time_periods[t]]
    wf_pt_post = wf_pt[wf_pt["daynum"] == time_periods[t+1]]
    wf_pt_post['Y1-Y0'] = wf_pt_post[out_var].values - wf_pt_pre[out_var].values
    wf_pt_post.reset_index(inplace=True)
    
    outcome_pt = wf_pt_post['Y1-Y0']
    treatment_pt = wf_pt_post['treat']
    confounders_pt = wf_pt_post[weather]
    # NOTE: this doesn't make sense bc the coefficient will always be 0
    Q0_pt,Q1_pt=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders_pt, y=outcome_pt, A=treatment_pt, n_splits=10, output_type="continuous")
    print("t: ", time_periods[t], "->", time_periods[t+1], "\t", np.mean(Q1_pt - Q0_pt))
    pairwise_time_diffs.append(np.mean(Q1_pt - Q0_pt))


t:  8401 -> 8402 	 0.0
t:  8402 -> 8403 	 0.0
t:  8403 -> 8404 	 0.0
t:  8404 -> 8405 	 0.0
t:  8405 -> 8406 	 0.0
t:  8406 -> 8407 	 0.0
t:  8407 -> 8408 	 0.0
t:  8408 -> 8409 	 0.0
t:  8409 -> 8410 	 0.0
t:  8410 -> 8411 	 0.0
t:  8411 -> 8412 	 0.0
t:  8412 -> 8413 	 0.0
t:  8413 -> 8414 	 0.0
t:  8414 -> 8415 	 0.0
t:  8415 -> 8416 	 0.0
t:  8416 -> 8417 	 0.0
t:  8417 -> 8418 	 0.0
t:  8418 -> 8419 	 0.0
t:  8419 -> 8420 	 0.0
t:  8420 -> 8421 	 0.0
t:  8421 -> 8422 	 0.0
t:  8422 -> 8423 	 0.0
t:  8423 -> 8424 	 0.0
t:  8424 -> 8425 	 0.0
t:  8425 -> 8426 	 0.0
t:  8426 -> 8427 	 0.0
t:  8427 -> 8428 	 0.0
t:  8428 -> 8429 	 0.0
t:  8429 -> 8430 	 0.0
t:  8430 -> 8431 	 0.0
t:  8431 -> 8432 	 0.0
t:  8432 -> 8433 	 0.0
t:  8433 -> 8434 	 0.0
t:  8434 -> 8435 	 0.0
t:  8435 -> 8436 	 -10.130868148113361
t:  8436 -> 8437 	 12.819396864298037
t:  8437 -> 8438 	 6.713918999099076
t:  8438 -> 8439 	 6.396595591412179
t:  8439 -> 8440 	 22.88629774082264
t:  8440 -> 8441 	 19.85604813